# 05g - Vertex AI > Training > Hyperparameter Tuning Jobs

### 05 Series Overview
Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the `05` notebook, the model training happened directly in the notebook.  The models were then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `05a-05i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI as custom training jobs.  These jobs will be demonstrated as:

-  Custom Job from a python script (`05a`), python source distribution (`05b`), and custom container (`05c`)
-  Training Pipeline that trains and saves models from a python script (`05d`), python source distribution (`05e`), and custom container (`05f`)
-  Hyperparameter Tuning Jobs from a python script (`05g`), python source distribution (`05h`), and custom container (`05i`)

### This Notebook (`05g`): An extension of `05a` with Hyperparameter Tuning
This notebook trains the same Tensorflow Keras model from `05` by first modifying and saving the training code to a python script (same as `05a`).

The script is then used as an input for a Vertex AI > Training > Custom Job that is also assigned compute resources and a [pre-built container for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for executing the training in a managed service. This is done with [Vertex AI Python SDK](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#) using the class [`aiplatform.CustomJob.from_local_script()`](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#google.cloud.aiplatform.CustomJob.from_local_script).

The Custom Job is then used as the input for a Vertex AI > Training > Hyperparameter Tuning Job.  This runs and manages the tuning loops for the number of trials in each loop, collects the metric(s) and manages the parameters with the selected search algorithm for parameter modification.  This is done with the [Vertex AI Python SDK](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#) using the class [`aiplatform.HyperparameterTuningJob()`](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#google.cloud.aiplatform.HyperparameterTuningJob).

The training can be reviewed with Vertex AI's managed Tensorboard under Vertex AI > Experiments > Experiments, or by clicking on the `05g...` job under Vertex AI > Training > Hyperparameters Tuning Jobs and then clicking the 'Open Tensorboard' link. Each set of hyperparameters tried will show up as a separate run of the model training.  Tensorboard has section called Hparams that makes comparing these runs easier and the setup and use of this feature is in the next notebooks of the series `05h` and `05i`.

<img src="architectures/overview/Training.png">

### Prerequisites:
-  01 - BigQuery - Table Data Source
-  Understanding:
    -  05 - Vertex AI > Notebooks - Models Built in Notebooks with Tensorflow
        -  Contains a more granular review of the Tensorflow model training

### Overview:
-  Setup
-  Connect to Tensorboard instance from 05
-  Create a `train.py` Python script that recreates the local training in 05
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Custom training job with aiplatform.CustomJob.from_local_script
   -  Hyperparameter tuning job with aiplatform.HyperparameterTuningJob
      -  Run job with .run
   -  Upload best Model to Vertex AI with aiplatform.Model.upload
   -  Create Endpoint with Vertex AI with aiplatform.Endpoint.create
      -  Deploy model to endpoint with .deploy 
-  Online Prediction demonstrated using Vertex AI Endpoint with deployed model
   -  Get records to score from BigQuery table
   -  Prediction with aiplatform.Endpoint.predict
   -  Prediction with REST
   -  Prediction with gcloud (CLI)

### Resources:
-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  Containers for training (Pre-Built)
   -  [Overview](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
   -  [List](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)
-  Vertex AI Hyperparameter Tuning
   -  [Overview of Hyperparameter Tuning](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)
   -  [Using Hyperparameter Tuning](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning)


---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/05g_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/05g_console.png">

---
## Setup

inputs:

In [1]:
REGION = 'us-central1'
PROJECT_ID='statmike-demo3'
DATANAME = 'fraud'
NOTEBOOK = '05g'

# Resources
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest'
DEPLOY_IMAGE ='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters
EPOCHS = 10
BATCH_SIZE = 100

packages:

In [2]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [4]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

In [5]:
# Give service account roles/storage.objectAdmin permissions
# Console > IMA > Select Account <projectnumber>-compute@developer.gserviceaccount.com > edit - give role
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'715288179162-compute@developer.gserviceaccount.com'

environment:

In [6]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get Vertex AI Experiments Tensorboard Instance Name
[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) has managed [Tensorboard](https://www.tensorflow.org/tensorboard) instances that you can track Tensorboard Experiments (a training run or hyperparameter tuning sweep).  

The training job will show up as an experiment for the Tensorboard instance and have the same name as the training job ID.

This code checks to see if a Tensorboard Instance has been created in the project, retrieves it if so, creates it otherwise:

In [7]:
tb = aiplatform.Tensorboard.list(filter=f'display_name={DATANAME}')
if tb:
    tb = tb[0]
else:
    tb = aiplatform.Tensorboard.create(display_name = DATANAME, labels = {'notebook':f'{DATANAME}'})

In [8]:
tb.resource_name

'projects/715288179162/locations/us-central1/tensorboards/6212451803166932992'

---
## Training

### Assemble Python File for Training

Create the main python trainer file as `/train.py`:

In [9]:
%%writefile {DIR}/train.py

# package import
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf
from google.cloud import bigquery
import argparse
import os
import sys
import hypertune

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--epochs', dest = 'epochs', default = 10, type = int, help = 'Number of Epochs')
parser.add_argument('--batch_size', dest = 'batch_size', default = 32, type = int, help = 'Batch Size')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str, nargs='*')
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--dataname', dest = 'dataname', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--notebook', dest = 'notebook', type=str)
# hyperparameters
parser.add_argument('--lr',dest='learning_rate', required=True, type=float, help='Learning Rate')
parser.add_argument('--m',dest='momentum', required=True, type=float, help='Momentum')
args = parser.parse_args()

# built in parameters for data source:
PROJECT_ID = args.project_id
DATANAME = args.dataname
REGION = args.region
NOTEBOOK = args.notebook

# clients
bigquery = bigquery.Client(project = PROJECT_ID)

# get schema from bigquery source
query = f"SELECT * FROM {DATANAME}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{DATANAME}_prepped'"
schema = bigquery.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bigquery.query(query = f'SELECT DISTINCT {args.var_target} FROM {DATANAME}.{DATANAME}_prepped WHERE {args.var_target} is not null').to_dataframe()
nclasses = nclasses.shape[0]

# Make a list of columns to omit
OMIT = args.var_omit + ['splits']

# use schema to prepare a list of columns to read from BigQuery
selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

# all the columns in this data source are either float64 or int64
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in schema[~schema.column_name.isin(OMIT)].data_type.tolist()]

# remap input data to Tensorflow inputs of features and target
def transTable(row_dict):
    target=row_dict.pop(args.var_target)
    target = tf.one_hot(tf.cast(target,tf.int64), nclasses)
    target = tf.cast(target, tf.float32)
    return(row_dict, target)

# function to setup a bigquery reader with Tensorflow I/O
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{PROJECT_ID}",
        project_id = PROJECT_ID,
        table_id = f"{DATANAME}_prepped",
        dataset_id = DATANAME,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training

train = bq_reader('TRAIN').parallel_read_rows().prefetch(1).map(transTable).shuffle(args.batch_size*10).batch(args.batch_size)
validate = bq_reader('VALIDATE').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)
test = bq_reader('TEST').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)

# Logistic Regression

# model input definitions
feature_columns = {header: tf.feature_column.numeric_column(header) for header in selected_fields if header != args.var_target}
feature_layer_inputs = {header: tf.keras.layers.Input(shape = (1,), name = header) for header in selected_fields if header != args.var_target}

# feature columns to a Dense Feature Layer
feature_layer_outputs = tf.keras.layers.DenseFeatures(feature_columns.values())(feature_layer_inputs)

# batch normalization then Dense with softmax activation to nclasses
layers = tf.keras.layers.BatchNormalization()(feature_layer_outputs)
layers = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax)(layers)

# the model
model = tf.keras.Model(
    inputs = feature_layer_inputs,
    outputs = layers
)
opt = tf.keras.optimizers.SGD() #SGD or Adam
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(
    optimizer = opt,
    loss = loss,
    metrics = ['accuracy', tf.keras.metrics.AUC(curve='PR')]
)

# setup tensorboard logs and train
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'], histogram_freq=1)
history = model.fit(train, epochs = args.epochs, callbacks = [tensorboard_callback], validation_data = validate)

# output the model save files
model.save(os.getenv("AIP_MODEL_DIR"))

# report hypertune info back to Vertex AI Training > Hyperparamter Tuning Job
hpt = hypertune.HyperTune()
hpt.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag = 'loss',
    metric_value = history.history['loss'][-1],
    global_step = 1)

Writing temp/05g/train.py


### Setup Training Job

In [10]:
CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE),
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--dataname=" + DATANAME,
    "--region=" + REGION,
    "--notebook=" + NOTEBOOK
]

In [11]:
customJob = aiplatform.CustomJob.from_local_script(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    script_path = f"{DIR}/train.py",
    container_uri = TRAIN_IMAGE,
    args = CMDARGS,
    requirements = ['tensorflow_io'],
    replica_count = 1,
    machine_type = TRAIN_COMPUTE,
    accelerator_count = 0,
    base_output_dir = f"{URI}/{TIMESTAMP}",
    staging_bucket = f"{URI}/{TIMESTAMP}",
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.utils.source_utils:Training script copied to:
gs://statmike-demo3/fraud/models/05g/20220311133230/aiplatform-2022-03-11-13:37:02.109-aiplatform_custom_trainer_script-0.1.tar.gz.


### Setup Hyperparameter Tuning Job

In [12]:
METRIC_SPEC = {
    "loss": "minimize"
}

PARAMETER_SPEC = {
    "lr": aiplatform.hyperparameter_tuning.DoubleParameterSpec(min=0.001, max=0.1, scale="log"),
    "m": aiplatform.hyperparameter_tuning.DoubleParameterSpec(min=1e-7, max=0.9, scale="linear")
}

In [13]:
htJob = aiplatform.HyperparameterTuningJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    custom_job = customJob,
    metric_spec = METRIC_SPEC,
    parameter_spec = PARAMETER_SPEC,
    max_trial_count = 20,
    parallel_trial_count = 5,
    search_algorithm = None,
    labels = {'notebook':f'{NOTEBOOK}'}
)

### Run Training Job

In [14]:
htJob.run(
    service_account = SERVICE_ACCOUNT,
    tensorboard = tb.resource_name
)

INFO:google.cloud.aiplatform.jobs:Creating HyperparameterTuningJob
INFO:google.cloud.aiplatform.jobs:HyperparameterTuningJob created. Resource name: projects/715288179162/locations/us-central1/hyperparameterTuningJobs/5856367265930280960
INFO:google.cloud.aiplatform.jobs:To use this HyperparameterTuningJob in another session:
INFO:google.cloud.aiplatform.jobs:hpt_job = aiplatform.HyperparameterTuningJob.get('projects/715288179162/locations/us-central1/hyperparameterTuningJobs/5856367265930280960')
INFO:google.cloud.aiplatform.jobs:View HyperparameterTuningJob:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5856367265930280960?project=715288179162
INFO:google.cloud.aiplatform.jobs:View Tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+715288179162+locations+us-central1+tensorboards+6212451803166932992+experiments+5856367265930280960
INFO:google.cloud.aiplatform.jobs:HyperparameterTuningJob projects/715288179162/locations/

In [15]:
# if trial.state.name == 'SUCCEEDED'
losses = [trial.final_measurement.metrics[0].value if trial.state.name == 'SUCCEEDED' else 1 for trial in htJob.trials]
losses

[0.004346075467765331,
 0.0046480027958750725,
 0.005477451719343662,
 0.004930521361529827,
 0.005399656947702169,
 0.004757932852953672,
 0.005196845158934593,
 0.004814615473151207,
 0.00460150046274066,
 0.004234418738633394,
 0.0045186979696154594,
 0.004956488031893969,
 0.004824720323085785,
 0.004392147995531559,
 0.0050363983027637005,
 0.0041735367849469185,
 0.0044417232275009155,
 0.004892314318567514,
 0.004180781543254852,
 0.004246748052537441]

In [16]:
best = htJob.trials[losses.index(min(losses))]
best

id: "16"
state: SUCCEEDED
parameters {
  parameter_id: "lr"
  value {
    number_value: 0.009700301142833653
  }
}
parameters {
  parameter_id: "m"
  value {
    number_value: 0.43372487913165925
  }
}
final_measurement {
  step_count: 1
  metrics {
    metric_id: "loss"
    value: 0.0041735367849469185
  }
}
start_time {
  seconds: 1647008136
  nanos: 475930672
}
end_time {
  seconds: 1647008708
}

## Review Results in Tensorboard
- In Console, go to Vertex AI > Training > Hyperparameter Tuning Jobs
- Click the `05g_` job
- Click `Open Tensorboard` at the top of the screen

<img src="architectures/notebooks/05g_screenshots/console_hpjob.png">

<img src="architectures/notebooks/05g_screenshots/console_hpjob2.png">

<img src="architectures/notebooks/05g_screenshots/tb_scalars.png">

<img src="architectures/notebooks/05g_screenshots/tb_histograms.png">

<img src="architectures/notebooks/05g_screenshots/tb_graphs.png">

---
## Serving

### Upload The Model

In [17]:
model = aiplatform.Model.upload(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = f"{URI}/{TIMESTAMP}/{best.id}/model",
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/715288179162/locations/us-central1/models/4199628642755543040/operations/3707575817568518144
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/715288179162/locations/us-central1/models/4199628642755543040
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/715288179162/locations/us-central1/models/4199628642755543040')


In [18]:
model.display_name

'05g_fraud_20220311133230'

### Create An Endpoint

In [19]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/715288179162/locations/us-central1/endpoints/761293054979080192/operations/9174945765196300288
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/715288179162/locations/us-central1/endpoints/761293054979080192
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/715288179162/locations/us-central1/endpoints/761293054979080192')


In [20]:
endpoint.display_name

'05g_fraud_20220311133230'

### Deploy Model To Endpoint

In [21]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/715288179162/locations/us-central1/models/4199628642755543040 to Endpoint : projects/715288179162/locations/us-central1/endpoints/761293054979080192
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/715288179162/locations/us-central1/endpoints/761293054979080192/operations/4518223750495207424
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/715288179162/locations/us-central1/endpoints/761293054979080192


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [22]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [23]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,7148,1.156386,0.193513,0.242220,0.660729,0.236144,0.311471,-0.088420,0.057844,1.123405,...,-0.051662,-0.262183,0.477870,0.556403,-0.046953,-0.021878,0.0,0,0eddc3ef-a61b-4fba-a3ab-0ed9a726dcf0,TEST
1,76311,-0.186529,0.545755,2.432618,3.266129,-0.784549,3.167033,-2.460489,-1.830983,0.389492,...,-0.400380,-1.265280,1.231000,0.749402,0.147862,0.187856,0.0,0,b1111e03-a559-4eb4-ab32-e3aea0072ef7,TEST
2,125139,1.879049,0.212473,-0.085529,3.554091,0.205505,1.188395,-0.672662,0.375249,-0.494351,...,0.131433,0.256023,-0.135450,0.048878,0.003082,-0.042219,0.0,0,0a0f4b69-01ee-436e-ae52-02237cd6433e,TEST
3,51632,1.264050,0.182193,0.020910,0.478060,-0.037823,-0.490973,0.166690,-0.130607,-0.157200,...,-0.167644,0.075563,0.698539,0.556361,-0.052595,-0.011799,0.0,0,ed678d6e-8dea-4d45-92b7-74e7eba22402,TEST


In [24]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
#newob

In [25]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [26]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[[0.999956846, 4.31437838e-05]], deployed_model_id='6534256866384412672', explanations=None)

In [27]:
prediction.predictions[0]

[0.999956846, 4.31437838e-05]

In [28]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [29]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [30]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.999956846,
      4.31437838e-05
    ]
  ],
  "deployedModelId": "6534256866384412672",
  "model": "projects/715288179162/locations/us-central1/models/4199628642755543040",
  "modelDisplayName": "05g_fraud_20220311133230"
}


### Get Predictions: gcloud (CLI)

In [31]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999956846, 4.31437838e-05]]


---
## Remove Resources
see notebook "99 - Cleanup"